In [1]:
from bs4 import BeautifulSoup
import json
import urllib3

In [2]:
url = 'https://nothai.com/menu/'

In [3]:
http = urllib3.PoolManager()
response = http.request("GET", url)
html_doc = response.data.decode('utf-8')

In [4]:
soup = BeautifulSoup(html_doc)
menu = soup.find_all("section", {"class": "menu-main-parent-row"})

In [5]:
menu

[<section class="l-section wpb_row us_custom_6e74c4b0 menu-main-parent-row height_small"><div class="l-section-h i-cf"><div class="g-cols vc_row via_flex valign_top type_default stacking_default"><div class="vc_col-sm-12 wpb_column vc_column_container"><div class="vc_column-inner"><div class="wpb_wrapper"><h2 class="w-text us_custom_f87f6ed2"><span class="w-text-h"><span class="w-text-value">Select your Entree</span></span></h2><h3 class="w-text us_custom_0dd32b33"><span class="w-text-h"><span class="w-text-value">Noodles</span></span></h3><div class="g-cols wpb_row via_flex valign_top type_default stacking_default"><div class="vc_col-sm-12 wpb_column vc_column_container"><div class="vc_column-inner"><div class="wpb_wrapper"><div class="w-hwrapper align_justify valign_middle" style="--hwrapper-gap:0.2rem"><div class="wpb_text_column"><div class="wpb_wrapper"><p>1. Pad Thai!*</p>
 </div></div><div class="w-image align_none"><div class="w-image-h"><img alt="gluten free" class="attachment

In [6]:
food_types = []
for i in range(len(menu)):
    if i == 0:
        food_types.append(menu[i].find_all("span")[2].text)
    else:
        food_types.append(menu[i].find("span").text)
        
food_types

with open('food_types.txt', 'w') as f:
    for food in food_types:
        f.write(f"{food}\n")